<a href="https://colab.research.google.com/github/maskot1977/AdvancedTheoryOfPharmacoinformaticsSimulation/blob/NUBzFryTHxL2/%E3%83%95%E3%82%A1%E3%83%BC%E3%83%9E%E3%82%B3%E3%82%A4%E3%83%B3%E3%83%95%E3%82%A9%E3%83%9E%E3%83%86%E3%82%A3%E3%82%AF%E3%82%B7%E3%82%B7%E3%83%9F%E3%83%A5%E3%83%AC%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3%E7%89%B9%E8%AB%96%E7%AC%AC3%E5%9B%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ファーマコインフォマティクスシミュレーション特論

小寺正明

# 第3回　化学構造情報処理

第1回、第2回は純粋に機械学習の諸問題について解説しましたが、第3回は話題を変えて、化学構造をコンピュータ上で処理する方法について解説します。

# RDKitのインストール

ここでは、プログラミング言語 Python で化学構造を取り扱うときによく用いられる RDKit を使います。まずはインストールから。理由は分かりませんが、次のスクリプトは時々失敗します。５分程度待っても終了しない場合は、その後いつまで待っても終了しないことがあります。そのようなときは、一旦停止して再度実行するか、再起動して再度実行してください。

In [ ]:
# RDKit installer のインストール
!pip install git+https://github.com/maskot1977/rdkit_installer.git

# RDKitのインストール
from rdkit_installer import install
install.from_miniconda(rdkit_version="2020.09.1")

# 化合物データのダウンロード

以下のように化合物データをダウンロードします。このデータは、PCCDBというデータベースからいくつかの化合物をピックアップしたものです。

In [ ]:
import urllib.request

# 化合物データのダウンロード
url = "https://raw.githubusercontent.com/maskot1977/tmd2020/main/data/data_1.csv"
urllib.request.urlretrieve(url, url.split("/")[-1])

# pandasのインポートとデータ読み込み

読み込んだ化合物データの内容は次のようにして確認できます。この中で、「PCCDB-ID」はデータベース中の化合物のID、「Open Babel SMILES」は化合物構造を「SMILES記法（スマイルスきほう、英語: simplified molecular input line entry system）」で表現したもの、残りの列はそれぞれの分子の様々な物性値です。

In [ ]:
import pandas as pd

# csvからのデータ読み込み
df = pd.read_csv('data_1.csv')
df

# RDKitを使った化学フォーマット変換

化合物構造の表現方法はSMILES以外にも色んなフォーマットがあります。RDKit は、それらのフォーマットを相互に変換する機能があります。

参考記事：https://rdkit.org/docs_jp/Getting_Started_with_RDKit_in_Python_jp.html

まずは、次のSMILESで表される分子を他のフォーマットに変換してみましょう。

In [ ]:
# １つめのSMILES
smile = df['Open Babel SMILES'][0]
smile

まずは SMILES を RDKit の Mol オブジェクトに変換します。

In [ ]:
from rdkit import Chem

# SMILES を RDKit Mol に変換
mol = Chem.MolFromSmiles(smile)
mol

Mol オブジェクトにすると、たとえば名前を付けるとか、様々な属性を付与できます。

In [ ]:
# 名前をつける
mol.SetProp("_Name", "Satan Miracle Special Ultra Super Megaton Punch")

次のようにして InChI に変換できます。

In [ ]:
# InChI に変換
inchi = Chem.MolToInchi(mol)
inchi

次のようにして、化合物を MolBlock 形式に変換できます。

In [ ]:
# MolBlock形式に変換
molblock = Chem.MolToMolBlock(mol)
print(molblock)

MolBlock はデフォルトでは水素原子を明記しませんが、明記する書き方に変更もできます。

In [ ]:
# 水素原子を明記する
mol = Chem.AddHs(mol)
molblock = Chem.MolToMolBlock(mol)
print(molblock)

水素原子を省略した形に戻すこともできます。

In [ ]:
# 水素原子を省略する
mol = Chem.RemoveHs(mol)
molblock = Chem.MolToMolBlock(mol)
print(molblock)

次のようにすれば、MolBlock形式のファイルを保存できます。

In [ ]:
# ファイルに出力する
filename = 'SatanMiracleSpecialUltraSuperMegatonPunch.mol'
print(Chem.MolToMolBlock(mol), file=open(filename, 'w+'))

# 描画

分子構造を取り扱ってはみましたが、やはり、どんな分子なのか「絵」を見てみたいですよね。

## １分子の描画

次のようにすれば分子構造を描画できます。

In [ ]:
from rdkit.Chem import Draw

smile = df['Open Babel SMILES'][0]
mol = Chem.MolFromSmiles(smile)
mol.SetProp("_Name", smile)

# ノートブック上に描画
Draw.MolToImage(mol, legend=mol.GetProp("_Name"))

次のようにして、画像ファイルを保存できます。

In [ ]:
# ファイルに出力する
filename = 'SatanMiracleSpecialUltraSuperMegatonPunch.png'
Chem.Draw.MolToFile(mol, filename, legend=mol.GetProp("_Name"))

## 構造式描画をもっと詳しく

MolBlock などのファイルフォーマットを詳細に見ていると、何番目の原子がどれなのか分からなくなることがあります。それが分かるように図示してみましょう。ただし、この方法では、原子が 0-index で表記されることに注意してください。

参考記事： https://future-chem.com/rdkit-2ddraw-svg/

In [ ]:
# どの原子にどの番号がついているか表示したい
from IPython.display import SVG
from rdkit.Chem.Draw import rdMolDraw2D

view = rdMolDraw2D.MolDraw2DSVG(300,350)
view.SetFontSize(0.9 * view.FontSize())
tm = rdMolDraw2D.PrepareMolForDrawing(mol)

option = view.drawOptions()
for atomidx, atom in enumerate(mol.GetAtoms()):
    symbol = mol.GetAtomWithIdx(atomidx).GetSymbol()
    option.atomLabels[atomidx] = symbol + str(atomidx)

view.DrawMolecule(tm, legend=mol.GetProp("_Name"))
view.FinishDrawing()
# 描画
svg = view.GetDrawingText()
SVG(svg)

次のようにすれば、指定した原子をハイライトできます。原子は 0-index で指定することにご注意ください。

In [ ]:
# 指定した原子をハイライトしたい
from IPython.display import SVG
from rdkit.Chem.Draw import rdMolDraw2D

colors = {6: (1,0,0), 8: (0,1,0), 10: (0, 0, 1)}
radii = {6: 0.25, 8: 0.50, 10: 0.2}

view = rdMolDraw2D.MolDraw2DSVG(300,350)
view.SetFontSize(0.9 * view.FontSize())
tm = rdMolDraw2D.PrepareMolForDrawing(mol)
option = view.drawOptions()
option.multipleBondOffset=0.07
option.padding=0.11
option.legendFontSize=20
view.DrawMolecule(tm,
                  highlightAtoms=colors.keys(),
                  highlightAtomColors=colors,
                  highlightAtomRadii=radii,
                  legend=mol.GetProp("_Name")
                  )
view.FinishDrawing()
# 描画
svg = view.GetDrawingText()
SVG(svg)

## 複数分子の描画

次のようにすれば、ライブラリ中にある複数の分子を一気に描画できます。

In [ ]:
mols = [] # 複数の RDKit Mol を格納するリスト
for smile, id in zip(df['Open Babel SMILES'], df['PCCDB-ID']):
    mol = Chem.MolFromSmiles(smile) # SMILES を RDKit Mol に変換
    mol.SetProp("_Name", str(id)) # 名前をつける
    mols.append(mol) # リストに格納する

In [ ]:
x_start = 0 # 表示したい最初の化合物
x_end = 100 # 表示したい最後の化合物の次
img = Draw.MolsToGridImage(
    mols[x_start:x_end], # スライスで範囲を指定
    molsPerRow=10, # 列の数
    subImgSize=(100,100), # 画像サイズ
    legends=[x.GetProp("_Name") for x in mols[x_start:x_end]], # 注釈
    )
display(img)

次のようにすれば画像データを保存できます。

In [ ]:
# ファイルに出力
img.save('SatanMiracleSpecialUltraSuperMegatonPunch_grid.png')

# カノニカライズ（記述順序の標準化）

これまで、化学構造を SMILES で表記してきましたが、SMILES 表記をするためには、記述する原子の順序を決定しなければなりません。残念なことに、記述順序は、使うソフトウェアによって異なります。今回用いたデータベースは Open Babel というツールによって記述された SMILES を用いていますので、Open Babel によるカノニカライズ（記述順序の標準化）をされていると考えられます。これを RDKit によるカノニカライズに切り替えたら、どのくらい変わってしまうか確認してみましょう。

In [ ]:
cnt = 0
for smile in df['Open Babel SMILES']:
    mol = Chem.MolFromSmiles(smile) # SMILES を RDKit Mol Object に変換し
    new_smile = Chem.MolToSmiles(mol) # それを SMILES に戻す
    if smile != new_smile: # もとの SMILES と新しい SMILES が違っていたら表示する
        cnt += 1
        print(cnt, "\t", smile, "\t", new_smile)

残念なことに（？）、Open Babel による SMILES と RDKit による SMILES はだいぶ異なることが分かります。

# 電荷の中和

次に、化学構造の前処理について紹介します。まずは「電荷の中和」です。


In [ ]:
from rdkit.Chem import MolStandardize

uc = MolStandardize.charge.Uncharger()

smile = "[NH3+][C@@H](Cc1c[nH]cn1)C(=O)[O-]"
before = Chem.MolFromSmiles(smile)
after = uc.uncharge(before)
smile2 = Chem.MolToSmiles(after)

print(smile, "=>", smile2)
Draw.MolsToGridImage([before, after])

# 脱塩

化学構造の前処理、次は「脱塩」です。

In [ ]:
from rdkit.Chem.SaltRemover import SaltRemover

remover = SaltRemover(defnData="[Cl,Br,Na]")

smile = "[NH3+][C@@H](Cc1c[nH]cn1)C(=O)[O-].[Na+].[Cl-]"
before = Chem.MolFromSmiles(smile)
after = remover.StripMol(before)
smile2 = Chem.MolToSmiles(after)

print(smile, "=>", smile2)
Draw.MolsToGridImage([before, after])

# 一番大きい分子だけ残す

化学構造の前処理、最後に「一番大きい分子だけ残す」です。

In [ ]:
from rdkit.Chem import MolStandardize

lfc = MolStandardize.fragment.LargestFragmentChooser()

smile = "[NH3+][C@@H](Cc1c[nH]cn1)C(=O)[O-].[Na+].[Cl-].[O-]C(=O)CCC(=O)[O-]"
before = Chem.MolFromSmiles(smile)
after = lfc.choose(before)
smile2 = Chem.MolToSmiles(after)

print(smile, "=>", smile2)
Draw.MolsToGridImage([before, after])

# 前処理を全部やる

In [ ]:
from rdkit.Chem import MolStandardize
from rdkit.Chem.SaltRemover import SaltRemover

uc = MolStandardize.charge.Uncharger()
remover = SaltRemover(defnData="[Cl,Br,Na]")
lfc = MolStandardize.fragment.LargestFragmentChooser()

preprocessed_smiles = []
for smile in df['Open Babel SMILES']:
    mol = Chem.MolFromSmiles(smile)
    smile = Chem.MolToSmiles(mol)
    mol = uc.uncharge(mol)
    mol = remover.StripMol(mol)
    mol = lfc.choose(mol)
    new_smile = Chem.MolToSmiles(mol)
    preprocessed_smiles.append(new_smile)
    if smile != new_smile:
        print(smile, new_smile)

上のようにすると、preprocessed_smiles には前処理済みのSMILESが格納されます。今回のケースでは、Open Babel SMILES が RDKit SMILES に変換されるだけで、それ以外の前処理の効果は特になかったようです。

# 部分構造検索

次は、化学構造情報処理でよく使われる、部分構造検索をやってみましょう。例として、次の分子の部分構造を取り扱ってみます。

In [ ]:
from rdkit.Chem import Draw

smile = df['Open Babel SMILES'][0]
mol = Chem.MolFromSmiles(smile)
mol.SetProp("_Name", "Recoom Ultra Fighting Miracle Bomber")

# ノートブック上に描画
Draw.MolToImage(mol, legend=smile)

例えば、次のような部分構造パターンを検索したいとします。

In [ ]:
# 検索したい部分構造パターン
smart = 'CNC'
patt = Chem.MolFromSmarts(smart)
Draw.MolToImage(patt, legend=smart)

次のようにすれば、指定した分子中で見つかったパターンのうち最初のものの原子IDが得られます。

In [ ]:
# 指定した分子中で見つかったパターンのうち最初のもの
mol.GetSubstructMatch(patt)

その原子IDをハイライトした分子構造を図示してみましょう。

In [ ]:
# 指定した原子をハイライトしたい
from IPython.display import SVG
from rdkit.Chem.Draw import rdMolDraw2D

view = rdMolDraw2D.MolDraw2DSVG(300,350)
view.SetFontSize(0.9 * view.FontSize())
tm = rdMolDraw2D.PrepareMolForDrawing(mol)
option = view.drawOptions()
option.multipleBondOffset=0.07
option.padding=0.11
option.legendFontSize=20
view.DrawMolecule(tm,
                  highlightAtoms=mol.GetSubstructMatch(patt),
                  #highlightAtomColors=colors,
                  #highlightAtomRadii=radii,
                  legend=str(mol.GetSubstructMatch(patt))
                  )
view.FinishDrawing()
# 描画
svg = view.GetDrawingText()
SVG(svg)

次のようにすれば、指定した分子中で見つかったパターン全部の原子IDが得られます。

In [ ]:
# 指定した分子中で見つかったパターン全部
mol.GetSubstructMatches(patt)

その原子IDをハイライトした分子構造を図示してみましょう。

In [ ]:
# 指定した原子をハイライトしたい
matches = mol.GetSubstructMatches(patt)
tm = rdMolDraw2D.PrepareMolForDrawing(mol)
yoko = 640
tate = 280
n_col = 4
n_row = 2
view = rdMolDraw2D.MolDraw2DSVG(
    yoko,
    tate,
    int(yoko / n_col * 0.9),
    int(tate / n_row * 0.9)
    )

view.DrawMolecules([tm]*len(matches),
                   highlightAtoms=matches, 
                   legends=[",".join([str(n) for n in match]) for match in matches]
                   )
# 描画
view.FinishDrawing()
svg = view.GetDrawingText()
SVG(svg)

今度は、次のような部分構造パターンを、データベース中から検索することを考えてみましょう。

In [ ]:
# 検索したい部分構造パターン
smart = 'c1ccc2[nH]ccc2c1'
patt = Chem.MolFromSmarts(smart)
Draw.MolToImage(patt, legend=smart)

そのような部分構造を含む分子が、df で指定されたデータベース中に何個あるか数えてみます。

In [ ]:
mols = []
for smile, id in zip(df['Open Babel SMILES'], df['PCCDB-ID']):
    mol = Chem.MolFromSmiles(smile)
    mol.SetProp("_Name", str(id))
    if mol.HasSubstructMatch(patt): # パターンを含む分子のみ mols に格納する
        mols.append(mol)

len(mols) # 格納された分子の数

得られた分子とその部分構造を図示します。

In [ ]:
# 描画
matches = [mol.GetSubstructMatches(patt)[0] for mol in mols]

Draw.MolsToGridImage(
    mols,
    molsPerRow=5,
    subImgSize=(160, 160),
    highlightAtomLists=matches,
    legends=[x.GetProp("_Name") for x in mols]
             )

以上のようにして得られた分子は、慣れれば、どの部分構造がどう共通しているかが分かりやすいかもしれませんが、部分構造の形状によっては、よく見ないと、どこが共通構造なのか分かりにくい場合もあります。そこで、できるだけ見やすくなるように、部分構造が描画される向きをできるだけ揃えるような図示の仕方をしてみましょう。

In [ ]:
# 指定した部分構造の向きを揃えて描画したい
from rdkit.Chem import AllChem

mols = []
AllChem.Compute2DCoords(patt) # 部分構造の2D座標を計算する
for smile, id in zip(df['Open Babel SMILES'], df['PCCDB-ID']):
    mol = Chem.MolFromSmiles(smile)
    mol.SetProp("_Name", str(id))
    if mol.HasSubstructMatch(patt): # パターンを含む分子のみ mols に格納する
        AllChem.GenerateDepictionMatching2DStructure(mol, patt) # 向きを揃える
        mols.append(mol)

matches = [mol.GetSubstructMatches(patt)[0] for mol in mols]

len(mols) # 格納された分子の数

# 描画
Draw.MolsToGridImage(
    mols[x_start:x_end],
    molsPerRow=5,
    subImgSize=(160, 160),
    highlightAtomLists=matches,
    legends=[x.GetProp("_Name") for x in mols]
             )

## 部分構造検索の注意点

部分構造検索には、ちょっとした（重要な）注意点があります。たとえば次のような部分構造を考えてみましょう。

In [ ]:
# 検索したい部分構造パターン
smart = 'c1ccccc1'
patt = Chem.MolFromSmarts(smart)
Draw.MolToImage(patt, legend=smart)

これを SMARTS 文字列として検索すると

In [ ]:
ids1 = []
patt = Chem.MolFromSmarts('c1ccccc1') # SMARTSを使ったパターン
for smile, id in zip(df['Open Babel SMILES'], df['PCCDB-ID']):
    mol = Chem.MolFromSmiles(smile)
    mol.SetProp("_Name", str(id))
    if mol.HasSubstructMatch(patt):
        ids1.append(id)

print(len(ids1))

これを SMILES 文字列として検索すると

In [ ]:
ids2 = []
patt = Chem.MolFromSmiles('c1ccccc1') # SMILESを使ったパターン
for smile, id in zip(df['Open Babel SMILES'], df['PCCDB-ID']):
    mol = Chem.MolFromSmiles(smile)
    mol.SetProp("_Name", str(id))
    if mol.HasSubstructMatch(patt):
        ids2.append(id)

print(len(ids2))

両者の違いは何でしょうか？

In [ ]:
set(ids1) - set(ids2) # SMARTSとSMILESで結果が異なることがある

上記のIDの分子が、SMARTSとSMILESで結果が異なる分子ということです。

In [ ]:
# 結果が異なる例を表示
for id in set(ids1) - set(ids2):
    smile = list(df[df['PCCDB-ID'] == id]['Open Babel SMILES'])[0]
    print(smile)
    mol = Chem.MolFromSmiles(smile)
    display(Draw.MolToImage(mol))

この分子中の環構造の一つが、SMILESとSMARTSで判定が異なるということですね。芳香族判定は部分構造検索で意図した結果にならない場合がよくあります。それ以外の場合でも、意図した部分構造がきちんと取れているかどうかのチェックは大事です。

## 部分構造検索における立体化学

部分構造検索では立体化学も考慮できます。次の２つの結果を比較してみましょう。

In [ ]:
mols = []
patt = Chem.MolFromSmiles('C[C@H](N)CO')
for smile, id in zip(df['Open Babel SMILES'], df['PCCDB-ID']):
    mol = Chem.MolFromSmiles(smile)
    mol.SetProp("_Name", str(id))
    if mol.HasSubstructMatch(patt, useChirality=True): # useChirality=True
        mols.append(mol)

print(len(mols))

x_start = 0
x_end = 5
Draw.MolsToGridImage(
    mols[x_start:x_end],
    molsPerRow=5,
    subImgSize=(200,200),
    legends=[x.GetProp("_Name") for x in mols[x_start:x_end]]
             )

In [ ]:
mols = []
patt = Chem.MolFromSmiles('C[C@@H](N)CO')
for smile, id in zip(df['Open Babel SMILES'], df['PCCDB-ID']):
    mol = Chem.MolFromSmiles(smile)
    mol.SetProp("_Name", str(id))
    if mol.HasSubstructMatch(patt, useChirality=True): # useChirality=True
        mols.append(mol)

print(len(mols))

x_start = 0
x_end = 5
Draw.MolsToGridImage(
    mols[x_start:x_end],
    molsPerRow=5,
    subImgSize=(200,200),
    legends=[x.GetProp("_Name") for x in mols[x_start:x_end]]
             )

# ３D構造の生成

ここまで扱ってきた化学構造は、二次元構造です。つまり、描画のためのXY座標は計算されてありますが、あくまで描画のための２次元座標であって、Z座標は計算されていません（Z座標を記述する場所は設けられているのですが全てゼロになっています）。例を見てみましょう。

In [ ]:
from rdkit.Chem import Draw

smile = df['Open Babel SMILES'][0]
mol = Chem.MolFromSmiles(smile)
mol.SetProp("_Name", smile)

# ノートブック上に描画
Draw.MolToImage(mol, legend=mol.GetProp("_Name"))

Z座標が全てゼロであることを確認してください。

In [ ]:
print(Chem.MolToMolBlock(mol))

次のように、「水素原子を明示する」→「三次元化する」→「水素原子を省略する」という手順を踏むことで、三次元座標を計算することができます。

In [ ]:
mol = Chem.AddHs(mol)
AllChem.EmbedMolecule(mol)
mol = Chem.RemoveHs(mol)
print(Chem.MolToMolBlock(mol))

# 3Dの描画

以上のようにして三次元座標を計算した分子を見てみましょう。ここでは py3Dmol を使います。

参考記事：https://future-chem.com/py3dmol/

In [ ]:
!pip install py3Dmol

次のようにすると、３次元空間中の分子をぐるぐる弄ぶことができます。

In [ ]:
import py3Dmol
from rdkit.Chem.Draw import IPythonConsole

view = py3Dmol.view(width=600, height=400, linked=False, viewergrid=(1,1))
view.addModel(Chem.MolToMolBlock(mol), 'sdf', {'keepH': False})
view.setStyle({'stick': {}}, viewer=(0, 0))
#view.setStyle({'sphere': {}}, viewer=(0, 0))
#view.setStyle({'line': {'linewidth': 50}}, viewer=(0,0))
view.setBackgroundColor('#000000', viewer=(0,0))
view.show()

描画方法は次のようにいろいろあります。弄びましょう。

In [ ]:
mols = [] # 複数の RDKit Mol を格納するリスト
for smile, id in zip(df['Open Babel SMILES'], df['PCCDB-ID']):
    mol = Chem.MolFromSmiles(smile) # SMILES　を　RDKit Mol　に変換
    mol.SetProp("_Name", str(id)) # 名前をつける
    mols.append(mol) # リストに格納する
    
add_mols = []
for m in mols[:6]:
    m_h = Chem.AddHs(m)
    AllChem.EmbedMolecule(m_h, AllChem.ETKDGv2())
    add_mols.append(m_h)
color_scheme = ['cyanCarbon', 'blackCarbon', 'ssPyMOL', 'ssJmol', 'Jmol', 'default']
view = py3Dmol.view(width=680, height=400, viewergrid=(2,3), linked=False)
for (n, (i,j)), c in zip(enumerate([(a,b) for a in range(2) for b in range(3)]), color_scheme):
    mb = Chem.MolToMolBlock(add_mols[n])
    view.addModel(mb, 'sdf', {'keepH': False}, viewer=(i,j))
    view.setStyle({'stick': {'colorscheme': c}}, viewer=(i,j))
view.zoomTo()
view.show()

# Recap

Recapは、実験室でよく使われる反応を模倣した化学変換のセットを使って、分子を一連の合理的なフラグメントに分解します。

Lewell, X.Q.; Judd, D.B.; Watson, S.P.; Hann, M.M. “RECAP-Retrosynthetic Combinatorial Analysis Procedure: A Powerful New Technique for Identifying Privileged Molecular Fragments with Useful Applications in Combinatorial Chemistry” J. Chem. Inf. Comp. Sci. 38:511-22 (1998).

例として次の分子を用いて、Recap の挙動を確認してみましょう。

In [ ]:
smile = df['Open Babel SMILES'][0]
print(smile)
mol = Chem.MolFromSmiles(smile)
Draw.MolToImage(mol)

次のようにしてフラグメントに分解します。

In [ ]:
from rdkit.Chem import Recap

# Recap アルゴリズムによる化学フラグメント分解
hierarch = Recap.RecapDecompose(mol)
ks = hierarch.children
ks

得られたフラグメントは次のようにして図示できます。

In [ ]:
# 得られたフラグメントの描画
Draw.MolsToGridImage(
    [Chem.MolFromSmiles(k) for k in ks.keys()],
    molsPerRow=5,
    subImgSize=(200,200),
    legends=list(ks.keys())
             )

# BRICS

BRICSは合成的に利用可能な結合に沿って分子をフラグメント化するもう一つの方法です

Degen, J.; Wegscheid-Gerlach, C.; Zaliani, A; Rarey, M. "On the Art of Compiling and Using ‘Drug-Like’ Chemical Fragment Spaces." ChemMedChem 3:1503–7 (2008)

同じ分子をこの方法でフラグメント化してみましょう。.

In [ ]:
from rdkit.Chem import BRICS

# BRICS アルゴリズムによる化学フラグメント分解
fragments = BRICS.BRICSDecompose(mol)

# 得られたフラグメントの描画
Draw.MolsToGridImage(
    [Chem.MolFromSmiles(k) for k in list(fragments)],
    molsPerRow=5,
    subImgSize=(200,200),
    legends=list(fragments)
             )

分子の集合に対して全てのフラグメントのリストを生成することがとても簡単にできます

In [ ]:
# フラグメント化したい分子の集合を作る
mols = []
for smile, id in zip(df['Open Babel SMILES'], df['PCCDB-ID']):
    mol = Chem.MolFromSmiles(smile)
    mol.SetProp("_Name", str(id))
    mols.append(mol)

x_start = 0
x_end = 10
Draw.MolsToGridImage(
    mols[x_start:x_end],
    molsPerRow=5,
    subImgSize=(200,200),
    legends=[x.GetProp("_Name") for x in mols[x_start:x_end]]
             )

In [ ]:
# 分子の集合から、フラグメントの集合を得る
allfrags=set()
for mol in mols[x_start:x_end]:
    fragment = BRICS.BRICSDecompose(mol)
    allfrags.update(fragment)

print(len(allfrags)) # 得られたフラグメントの数

# フラグメントの集合の描画
Draw.MolsToGridImage(
    [Chem.MolFromSmiles(f) for f in allfrags],
    molsPerRow=5,
    subImgSize=(200,200),
    legends=list(allfrags)
             )

In [ ]:
# フラグメントの集合にBRICSのルールを適用して新しい分子を生成する
fragms = [Chem.MolFromSmiles(x) for x in allfrags]

products = []
for mol in BRICS.BRICSBuild(fragms):
    mol.UpdatePropertyCache(strict=False)
    products.append(mol)
    if len(products) >= 25:
        break

# 得られた新しい分子の描画
Draw.MolsToGridImage(
    products,
    molsPerRow=5,
    subImgSize=(200,200),
    legends=[Chem.MolToSmiles(mol) for mol in products]
             )

# 分子ごとの化学記述子

ここまで、分子構造を取り扱ってきましたが、機械学習などで取り扱うためには、分子を「数字の集合」として表現した方が取り扱いやすくなります。そのための方法として「化学記述子」があります。これには、分子ごとの化学記述子と、原子や結合ごとの化学記述子とがあります（普通は、化学記述子といえば分子ごとの化学記述子を指すことが多いようです）。まずは分子ごとの化学記述子を見ていきましょう。例として、この分子を取り扱います。

In [ ]:
smile = df['Open Babel SMILES'][0]
print(smile)
mol = Chem.MolFromSmiles(smile)
Draw.MolToImage(mol)

トポロジカル極性表面積

In [ ]:
from rdkit.Chem import Descriptors
Descriptors.TPSA(mol)

Crippenらによる原子ベースのLogP指標

In [ ]:
from rdkit.Chem import Descriptors
Descriptors.MolLogP(mol)

分子量

In [ ]:
from rdkit.Chem import Descriptors
Descriptors.MolWt(mol)

非水素原子の数

In [ ]:
from rdkit.Chem import Descriptors
Descriptors.HeavyAtomCount(mol)

水素結合アクセプターの数

In [ ]:
from rdkit.Chem import Descriptors
Descriptors.NumHAcceptors(mol)

水素結合ドナーの数

In [ ]:
from rdkit.Chem import Descriptors
Descriptors.NumHDonors(mol)

へテロ元素の数

In [ ]:
from rdkit.Chem import Descriptors
Descriptors.NumHeteroatoms(mol)

回転可能な結合数

In [ ]:
from rdkit.Chem import Descriptors
Descriptors.NumRotatableBonds(mol)

環の数

In [ ]:
from rdkit.Chem import Descriptors
Descriptors.RingCount(mol)

全炭素数におけるsp3炭素の割合

In [ ]:
from rdkit.Chem import Descriptors
Descriptors.FractionCSP3(mol)

以上、説明が比較的簡単な記述子について列挙しました。

## 計算可能な記述子を列挙してマトリクスを作る

説明が比較的簡単な記述子から、説明が面倒な記述子まで、全部でいくつあるかリストアップしてみます。

In [ ]:
desc_names = [x[0] for x in Descriptors._descList if x[0]]
print(desc_names)

In [ ]:
len(desc_names)

データベース上の全分子に対してこれらを計算して、マトリックスを作ってみましょう。

In [ ]:
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import MolStandardize
from rdkit.Chem.SaltRemover import SaltRemover

uc = MolStandardize.charge.Uncharger()
remover = SaltRemover(defnData="[Cl,Br,Na]")
lfc = MolStandardize.fragment.LargestFragmentChooser()
calc = MoleculeDescriptors.MolecularDescriptorCalculator(desc_names)

matrix = []
for smile, id in zip(df['Open Babel SMILES'], df['PCCDB-ID']):
    row = []
    row.append(id)
    mol = Chem.MolFromSmiles(smile)
    mol = uc.uncharge(mol)
    mol = remover.StripMol(mol)
    mol = lfc.choose(mol)
    row.append(Chem.MolToSmiles(mol))
    for d in calc.CalcDescriptors(mol):
        row.append(d)
    matrix.append(row)

desc_df = pd.DataFrame(matrix, columns=["ID", "SMILES"] + desc_names)
desc_df

以上のコードを毎回書くのもめんどくさいので、次のようなツールを私が作りました。

In [ ]:
from rdkit_installer.descriptors import calc_208descriptors

descriptors = calc_208descriptors(df['Open Babel SMILES'])
descriptors

RDKit で計算できる記述子は、実はまだまだありまして、説明が難しいのを入れると全部で400個ほどあります。これらをまとめて計算するツールも私が作りました。次のようにして使います。

In [ ]:
from rdkit_installer.descriptors import calc_400descriptors

descriptors = calc_400descriptors(df['Open Babel SMILES'])
descriptors

 RDKit のバージョンによって、計算可能な記述子の数は異なります。

# 原子ごとの物理化学的特徴

普通は「記述子」と呼ばないかもしれませんが、分子ではなく原子ごとに与えられる特徴量もあります。次の化合物を例にしてみましょう。

In [ ]:
smile = df['Open Babel SMILES'][0]
print(smile)
mol = Chem.MolFromSmiles(smile)
Draw.MolToImage(mol)

その代表例が Gasteiger法を用いた部分電荷 です。

In [ ]:
from rdkit.Chem import AllChem

AllChem.ComputeGasteigerCharges(mol)
for id, atom in enumerate(mol.GetAtoms()):
    print(id + 1, atom.GetSymbol(), atom.GetProp('_GasteigerCharge'))

薬学の世界では「ファーマコフォア」という概念があります。医薬品のターゲットとの相互作用に必要な特徴を持つ官能基群と、それらの相対的な立体配置も含めた（抽象的な）概念だそうです。RDKitでは次のように計算して取得できます。

In [ ]:
import os
from rdkit import RDConfig
from rdkit.Chem import ChemicalFeatures
fdefName = os.path.join(RDConfig.RDDataDir,'BaseFeatures.fdef')
factory = ChemicalFeatures.BuildFeatureFactory(fdefName)

for i, feat in enumerate(factory.GetFeaturesForMol(mol)):
    print(i, feat.GetFamily(), feat.GetType(), feat.GetAtomIds())

## SASA (solvent accessible surface area)

溶媒露出表面積は、分子ごとの記述子として取り扱うことができます。

In [ ]:
from rdkit.Chem import rdFreeSASA

mol = Chem.AddHs(mol)
AllChem.EmbedMolecule(mol)

radii = rdFreeSASA.classifyAtoms(mol)
rdFreeSASA.CalcSASA(mol, radii)

次のようにすれば、原子ごとの記述子として取り扱うこともできます。

In [ ]:
atoms = mol.GetAtoms()
for i in range(len(atoms)):
    print(i, atoms[i].GetSymbol(), atoms[i].GetProp("SASA"))

# 2Dファーマコフォアフィンガープリント

複数の化学的特徴と、フィーチャー間の2Dの(幾何学的、トポロジカル)距離とを合わせることで2Dファーマコフォアを作成できます。

In [ ]:
from rdkit import Chem
from rdkit.Chem import ChemicalFeatures
from rdkit.Chem.Pharm2D.SigFactory import SigFactory

fdefName = os.path.join(RDConfig.RDDataDir, 'BaseFeatures.fdef')
featFactory = ChemicalFeatures.BuildFeatureFactory(fdefName)
sigFactory = SigFactory(featFactory, minPointCount=2, maxPointCount=3)

# 距離をビン分割して、それぞれのファーマコフォアに一意の整数値のIDを割り当てる
sigFactory.SetBins([(0,2),(2,5),(5,8)])
sigFactory.Init()
sigFactory.GetSigSize()

In [ ]:
# 2Dファーマコフォアフィンガープリントの作成
from rdkit.Chem.Pharm2D import Generate
fp = Generate.Gen2DFingerprint(mol, sigFactory)
len(fp) # フィンガープリントの長さ

In [ ]:
fp.GetNumOnBits() # 立っているビットの数

In [ ]:
# 立っているビットの内訳
for bit in list(fp.GetOnBits()):
    print(bit, sigFactory.GetBitDescription(bit))

Gobbi & Poppinger 定義の２Dファーマコフォアを用いる

Gobbi, A. & Poppinger, D. "Genetic optimization of combinatorial libraries." Biotechnology and Bioengineering 61:47-54 (1998).

In [ ]:
from rdkit import Chem
from rdkit.Chem.Pharm2D import Gobbi_Pharm2D,Generate
fp = Generate.Gen2DFingerprint(mol, Gobbi_Pharm2D.factory)
len(fp) # フィンガープリントの長さ

In [ ]:
fp.GetNumOnBits() # 立っているビットの数

In [ ]:
# 立っているビットの内訳
for bit in list(fp.GetOnBits()):
    print(bit, Gobbi_Pharm2D.factory.GetBitDescription(bit))

# フィンガープリントを用いた類似分子検索

化学構造を数字で表現する方法としては他に、フィンガープリントというものがあります。フィンガープリントは一般的に、部分構造などの化学的特徴の有無を 1 か 0 かの bit vector として表す方法と、化学的特徴の個数を自然数の count vector として表す方法とがあります。次の分子を例として計算してみましょう。

In [ ]:
smile = df['Open Babel SMILES'][0]
print(smile)
query_mol = Chem.MolFromSmiles(smile)
Draw.MolToImage(query_mol)

フィンガープリントは、化合物データベース中から、クエリー（問い合わせ）分子と類似した化合物を検索する目的で当初使われました。さまざまな種類のフィンガープリントが開発され、さまざまな種類の類似性指標が開発されています。

In [ ]:
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
from rdkit.Chem.AtomPairs import Pairs
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem.AtomPairs import Torsions
from rdkit.Avalon import pyAvalonTools

# フィンガープリントいろいろ
get_fp = lambda mol: MACCSkeys.GenMACCSKeys(mol)
#get_fp = lambda mol: AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
#get_fp = lambda mol: AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024, useFeatures=True)
#get_fp = lambda mol: AllChem.GetMorganFingerprintAsBitVect(mol, 4, nBits=1024)
#get_fp = lambda mol: AllChem.GetMorganFingerprintAsBitVect(mol, 4, nBits=1024, useFeatures=True)
#get_fp = lambda mol: Pairs.GetAtomPairFingerprintAsBitVect(mol)
#get_fp = lambda mol: FingerprintMols.FingerprintMol(mol) #Topological Fingerprint
#get_fp = lambda mol: Torsions.GetTopologicalTorsionFingerprintAsIntVect(mol) 
#get_fp = lambda mol: pyAvalonTools.GetAvalonFP(mol)

# 類似性指標もいろいろ
metric = DataStructs.TanimotoSimilarity
#metric = DataStructs.DiceSimilarity
#metric = DataStructs.CosineSimilarity
#metric = DataStructs.SokalSimilarity
#metric = DataStructs.RusselSimilarity
#metric = DataStructs.KulczynskiSimilarity
#metric = DataStructs.McConnaugheySimilarity

例えば次のようにして、データベース中から類似化合物を検索することができます。

In [ ]:
query_fp = get_fp(query_mol)

mols = []
fps = [] # 複数のリンガープリントを格納するリスト
for smile, id in zip(df['Open Babel SMILES'], df['PCCDB-ID']):
    mol = Chem.MolFromSmiles(smile) # SMILES　を　RDKit Mol　に変換
    mol.SetProp("_Name", str(id)) # 名前をつける
    fp = get_fp(mol)
    fps.append(fp) # リストに格納する
    mols.append(mol)

results = []
for i, fp in enumerate(fps):
    results.append([DataStructs.FingerprintSimilarity(query_fp, fp, metric=metric), i])

searched_mols = []
similarity_values = []
for sim, i in reversed(sorted(results)):
    searched_mols.append(mols[i])
    similarity_values.append(sim)

x_start = 0 # 表示したい最初の化合物
x_end = 10 # 表示したい最後の化合物の次
Draw.MolsToGridImage(
    searched_mols[x_start:x_end], # スライスで範囲を指定
    molsPerRow=5, # 列の数
    subImgSize=(200,200), # 画像サイズ
    legends=[str(v) for v in similarity_values] # 注釈
             )

# 最大共通部分構造

化合物の類似性といえば、最大共通部分構造の大きさによって定義することも可能です。

次のようにすれば、複数の分子の最大共通部分構造を得ることができます。

In [ ]:
from rdkit.Chem import rdFMCS

mcs = rdFMCS.FindMCS([query_mol, mol], timeout=60)
mcs.numAtoms, mcs.numBonds # 最大共通部分構造に含まれる原子数、結合数

In [ ]:
mol_mcs = Chem.MolFromSmarts(mcs.smartsString)
mcs1 = query_mol.GetSubstructMatch(mol_mcs) 
mcs2 = mol.GetSubstructMatch(mol_mcs)
mcs1, mcs2 #それぞれで一致した原子番号の一覧

複数の分子から得られた最大共通部分構造の例です。

In [ ]:
Draw.MolsToGridImage(
    [query_mol, mol], 
    molsPerRow=2, 
    subImgSize=(400,400), 
    highlightAtomLists=[mcs1, mcs2]
    )

次のコードは、最大共通部分構造の大きさに基づいた類似性検索の例です。

In [ ]:
# 最大共通部分構造の大きさに基づいた類似性検索

mols = []
for smile, id in zip(df['Open Babel SMILES'], df['PCCDB-ID']):
    mol = Chem.MolFromSmiles(smile) # SMILES　を　RDKit Mol　に変換
    mol.SetProp("_Name", str(id)) # 名前をつける
    mols.append(mol)

results = []
for i, mol in enumerate(mols):
    mcs = rdFMCS.FindMCS([query_mol, mol], timeout=60)
    results.append([mcs.numAtoms, mcs.numBonds, i])

searched_mols = []
similarity_values = []
highlight_atom_lists = []
for n_atom, n_bond, i in reversed(sorted(results)):
    searched_mols.append(mols[i])
    similarity_values.append(str(n_atom) + "," + str(n_bond))
    mcs = rdFMCS.FindMCS([query_mol, mols[i]], timeout=60)
    mol_mcs = Chem.MolFromSmarts(mcs.smartsString)
    mcs2 = mols[i].GetSubstructMatch(mol_mcs)
    highlight_atom_lists.append(mcs2)

x_start = 0 # 表示したい最初の化合物
x_end = 10 # 表示したい最後の化合物の次
Draw.MolsToGridImage(
    searched_mols[x_start:x_end], # スライスで範囲を指定
    molsPerRow=5, # 列の数
    subImgSize=(160, 160), # 画像サイズ
    highlightAtomLists=highlight_atom_lists[x_start:x_end],
    legends=similarity_values # 注釈
             )


# 課題



1.   コンピューター上における化合物の表現方法にはどのようなものがあるか、共通点や相違点を明らかにしながら説明してください。
2.   フィンガープリントに基づいた構造検索と、最大共通部分構造に基づいた構造検索について、共通点や相違点を明らかにしながら説明してください。
3.   フィンガープリントに基づいた構造検索について、さまざまな種類のフィンガープリントと、さまざまな種類の類似性手法の組み合わせを試し、どの組み合わせが最も良さそうか検討してください。また、最大共通部分構造に基づいた構造検索とも比較し、良し悪しを検討してください。最後に、良さそうだと判断した理由も説明してください。


# 提出方法：

下記のいずれかの方法で提出してください。

*   Google Colaboratory 上で動作させたコードを ikemenmaskot@gmail.com に「共有」
*   Jupyter Notebook 上で動作させたコードを ipynb 形式または html 形式にして ikemenmaskot@gmail.com に「メール送信」



# 次回

第1回・第2回は機械学習の話題を取り扱い、第3回は化学構造情報処理の話題を取り扱いました。私の担当分の最終回（第4回）は、これまでの全ての話題を統合して、化学構造情報を用いた機械学習に取り組みます。
